In [1]:
import numpy as np
import pandas as pd
import math
import json

In [2]:
#считываем данные об оценках из файла, нулевой столбец удалаем за ненадобностью 
#(учитываем, что номер строки на 1 меньше, чем номер User'a, поэтому User 17 находится в 16 строке)
data = pd.read_csv('data.csv', delimiter=', ', engine='python')
kNN = 4
myUserIndex = 16

data = data.drop(columns=[data.columns[0]])
print(data)

    Movie 1  Movie 2  Movie 3  Movie 4  Movie 5  Movie 6  Movie 7  Movie 8  \
0        -1        4        1        5       -1       -1       -1        1   
1         4        5        2        3        3        4       -1        5   
2         4        2        5        4        1        5       -1        3   
3         5       -1        2       -1       -1        2        4        3   
4         4        3        3        3        5        3        3        4   
5         2       -1        3        3        3        3        4        2   
6         4        1        4        4        1        4        5        2   
7        -1        5        5        3        3        3        3        3   
8         1        2        3        1        5        3        3        1   
9         5       -1        1       -1        3        3        4        1   
10       -1        5       -1        1        5        2        4        2   
11        5        4        4       -1        3       -1        

In [3]:
#определяем среднюю оценку выбранного по заданию человека
myUserSum=0
myUserCount=0
for column in data:   
    if data[column][myUserIndex] != -1:
        myUserSum+=data[column][myUserIndex]
        myUserCount+=1
avg_ru=myUserSum/myUserCount
print(avg_ru)
    

3.0


In [4]:
sim = dict.fromkeys(list(data.index) , -1)


In [5]:
#определяем метрику сходства для всех людей по сравнению с нашим человеком
for i, row in data.iterrows():
    if (i == myUserIndex):
        continue
    sum_uivi = 0
    sum_ui2 = 0
    sum_vi2 = 0
    for film in row.keys():
        if (data[film][myUserIndex]!=-1) and (row[film]!=-1):
            sum_uivi += data[film][myUserIndex]*data[film][i]
            sum_ui2 += data[film][myUserIndex]*data[film][myUserIndex]
            sum_vi2 += data[film][i]*data[film][i]
    sim[i] = round(sum_uivi/(math.sqrt(sum_ui2)*math.sqrt(sum_vi2)), 3)
print(sim)

{0: 0.819, 1: 0.805, 2: 0.715, 3: 0.862, 4: 0.808, 5: 0.883, 6: 0.773, 7: 0.81, 8: 0.87, 9: 0.826, 10: 0.835, 11: 0.822, 12: 0.807, 13: 0.758, 14: 0.809, 15: 0.838, 16: -1, 17: 0.725, 18: 0.769, 19: 0.899, 20: 0.822, 21: 0.803, 22: 0.815, 23: 0.851, 24: 0.765, 25: 0.781, 26: 0.875, 27: 0.856, 28: 0.809, 29: 0.731, 30: 0.802, 31: 0.644, 32: 0.725, 33: 0.802, 34: 0.803, 35: 0.827, 36: 0.834, 37: 0.815, 38: 0.793, 39: 0.889}


In [6]:
#сортируем людей по наибольшей совместимости с нашим человеком и выбираем первых четырёх из них
kNN_sim = dict(sorted(list(sim.items()), key=lambda x: x[1], reverse=True)[:kNN])
print(kNN_sim)

{19: 0.899, 39: 0.889, 5: 0.883, 26: 0.875}


In [7]:
#определяем фильмы без оценок для человека с индексом 17
unrated_movies = {}
for column in data:   
    if data[column][myUserIndex] == -1:
        unrated_movies[column] = -1
print(unrated_movies)

{'Movie 3': -1, 'Movie 4': -1, 'Movie 7': -1, 'Movie 8': -1, 'Movie 11': -1, 'Movie 12': -1, 'Movie 18': -1, 'Movie 26': -1}


In [8]:
#считаем оценки для неоценённых фильмов по формуле
for movie in unrated_movies:
    sum_top = 0
    sum_bottom = 0
    for key in kNN_sim:       
        avg_rv = 0
        sum_rv = 0
        count_rv = 0
        for column in data:
            if data[column][key] != -1:
                sum_rv += data[column][key]
                count_rv += 1
        avg_rv = sum_rv/count_rv if count_rv!=0 else 0
        
        if data[movie][key] != -1:
            sum_top += kNN_sim[key]*(data[movie][key] - avg_rv)
            #print(data[movie][key], avg_rv, key)
            sum_bottom += kNN_sim[key]
    unrated_movies[movie] = round(avg_ru+sum_top/sum_bottom, 3)

print(unrated_movies)

{'Movie 3': 2.086, 'Movie 4': 2.906, 'Movie 7': 2.834, 'Movie 8': 2.174, 'Movie 11': 3.102, 'Movie 12': 4.086, 'Movie 18': 3.327, 'Movie 26': 3.087}


In [9]:
#считываем данные о дне и месте просмотра в датафреймы
dataDays = pd.read_csv('context_day.csv', delimiter=', ', engine='python')
dataDays = dataDays.drop(columns=[dataDays.columns[0]])
dataPlaces = pd.read_csv('context_place.csv', delimiter=', ', engine='python')
dataPlaces = dataPlaces.drop(columns=[dataPlaces.columns[0]])
weekend = ['Sat', 'Sun']
home = 'h'

In [10]:
#аналогично учтём влияние тех людей, которые посмотрели неоценённые фильмы дома и/или в выходные:
#по той же формуле со слайда 6 посчитаем отношения (SUM(sim_vu*(rvi_h - avg_rv_h))/SUM(sim_vu)) и 
#(SUM(sim_vu*(rvi_w - avg_rv_w))/SUM(sim_vu)), и добавим их к существующим оценкам для фильмов (unrated_movies)
#rvi_h - оценка фильма, который был посмотрен из дома
#avg_rv_h - средняя оценка фильмов, посмотренных из дома (для конкретного человека)
#rvi_h - оценка фильма, который был посмотрен на выходных
#avg_rv_h - средняя оценка фильмов, посмотренных на выходных (для конкретного человека)
#данные оценки будем считать так же только для наиболее схожих с моим человеком людей (kNN_sim)
#учитывать будем только оценки фильмов, которые были посмотрены 
#дома/на выходных (в зависимости от того, какое из отношений считаем)
#с учётом допольнительных показателей оценка фильма может привысить 5

new_unrated_movies = unrated_movies.copy()
for movie in unrated_movies:
    sum_top_h = 0
    sum_bottom_h = 0
    sum_top_w = 0
    sum_bottom_w = 0
    for key in kNN_sim:       
        avg_rv_h = 0
        sum_rv_h = 0
        count_rv_h = 0
        avg_rv_w = 0
        sum_rv_w = 0
        count_rv_w = 0
        for column in data:
            if data[column][key] != -1 and (dataDays[column][key]== 'Sat' or dataDays[column][key]== 'Sun'):
                sum_rv_w += data[column][key]
                count_rv_w += 1
            if data[column][key] != -1 and dataPlaces[column][key]== 'h':
                sum_rv_w += data[column][key]
                count_rv_w += 1
        avg_rv_h = sum_rv_h/count_rv_h if count_rv_h!=0 else 0
        avg_rv_w = sum_rv_w/count_rv_w if count_rv_w!=0 else 0
        
        if data[movie][key] != -1:
            if dataDays[column][key]== 'Sat' or dataDays[column][key]== 'Sun':
                sum_top_w += kNN_sim[key]*(data[movie][key] - avg_rv_w)
                sum_bottom_w += kNN_sim[key]
                #print("weekend",data[movie][key] - avg_rv_w, key, movie)
            if dataPlaces[column][key]== 'h':
                sum_top_h += kNN_sim[key]*(data[movie][key] - avg_rv_h)
                sum_bottom_h += kNN_sim[key]
                #print("home",data[movie][key] - avg_rv_h, key, movie)
    
    new_unrated_movies[movie] = round(unrated_movies[movie]+(sum_top_h/sum_bottom_h if sum_bottom_h != 0 else 0) + (sum_top_w/sum_bottom_w if sum_bottom_w != 0 else 0), 3)

print(new_unrated_movies)

{'Movie 3': 5.086, 'Movie 4': 2.906, 'Movie 7': 5.834, 'Movie 8': 6.174, 'Movie 11': 4.102, 'Movie 12': 9.086, 'Movie 18': 8.327, 'Movie 26': 6.087}


In [11]:
#лучший фильм для просмотра в выходные дома
bestHomeWeekendFilm = dict([max(new_unrated_movies.items(), key=lambda x: x[1])])
print(bestHomeWeekendFilm)

{'Movie 12': 9.086}


In [12]:
#генерируем json-файл с результатми работы программы
output = {
    'user': myUserIndex + 1,
    '1': unrated_movies,
    '2': bestHomeWeekendFilm
}
print(output)
with open('output.json', 'w') as file:
    json.dump(output, file, indent = 8)

{'user': 17, '1': {'Movie 3': 2.086, 'Movie 4': 2.906, 'Movie 7': 2.834, 'Movie 8': 2.174, 'Movie 11': 3.102, 'Movie 12': 4.086, 'Movie 18': 3.327, 'Movie 26': 3.087}, '2': {'Movie 12': 9.086}}


In [13]:
# SPARQL в коде 3.2
# определим названия фильмов, для которых находили оценки
movieNames = pd.read_csv('movie_names.csv', delimiter=', ', names = ['index', 'name'], engine='python')
movies = list(map(lambda x: x, unrated_movies.keys()))
names = []
for m in movies:
     names.append(movieNames[movieNames['index'] == m].values[0][1])
print(names)

['The Godfather: Part II', 'The Dark Knight', 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Lord of the Rings: The Fellowship of the Ring', 'Forrest Gump', 'The Matrix', 'The Usual Suspects']


In [24]:
# находим id фильмов на wikidata
wikiIds = {}
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from pandas import json_normalize
API_ENDPOINT = "https://www.wikidata.org/w/api.php"
for name in names:
    params = {
        'action' : 'wbsearchentities',
        'format' : 'json',
        'language' : 'en',
        'search': name
    }
    res = requests.get(API_ENDPOINT, params = params)
    wikiIds[name] = res.json()['search'][0]['id']
wikiIds

{'The Godfather: Part II': 'Q202326',
 'The Dark Knight': 'Q2695156',
 'The Lord of the Rings: The Return of the King': 'Q131074',
 'Pulp Fiction': 'Q104123',
 'The Lord of the Rings: The Fellowship of the Ring': 'Q127367',
 'Forrest Gump': 'Q134773',
 'The Matrix': 'Q83495',
 'The Usual Suspects': 'Q132351'}

In [28]:
# проходимся по всем id и для каждого выполняем sparql запрос к wikidata
# результат запроса получаем в json, нормализуем его (преобразовываем в табличный вид)
# films_studio['имя фильма'] - список, содержащий фильмы той же студии и ссылки на них в wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
films_studio = {}
for [name, wikiId] in list(wikiIds.items()):
    sparql_query = """
SELECT DISTINCT ?item ?itemLabel WHERE {
  ?film wdt:P272 ?studio FILTER(?film = wd:""" + wikiId + """) .
  ?item wdt:P31 wd:Q11424.
  ?item wdt:P272 ?studios FILTER(?studios = ?studio).
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

    sparql.setQuery(sparql_query)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    #print(results_df)
    if results_df.size != 0:
        ids = results_df[['item.value', 'itemLabel.value']]['item.value'].values
        films = results_df[['item.value', 'itemLabel.value']]['itemLabel.value'].values
        films_studio[name] = []
        for i in range(len(ids)):
            films_studio[name].append({ "id": ids[i], "Film": films[i] })
    else:
        films_studio[name] = []
films_studio['Forrest Gump'][:10]

[{'id': 'http://www.wikidata.org/entity/Q1198073', 'Film': 'Hud'},
 {'id': 'http://www.wikidata.org/entity/Q1143603',
  'Film': 'The Vagabond King'},
 {'id': 'http://www.wikidata.org/entity/Q1196432', 'Film': 'The Rainmaker'},
 {'id': 'http://www.wikidata.org/entity/Q1219315',
  'Film': 'Death Takes a Holiday'},
 {'id': 'http://www.wikidata.org/entity/Q1356395', 'Film': 'Alfie'},
 {'id': 'http://www.wikidata.org/entity/Q1167792',
  'Film': 'The Family Jewels'},
 {'id': 'http://www.wikidata.org/entity/Q1197663',
  'Film': 'The Disorderly Orderly'},
 {'id': 'http://www.wikidata.org/entity/Q1199439', 'Film': 'The Last Tycoon'},
 {'id': 'http://www.wikidata.org/entity/Q1206842', 'Film': 'The Devil Inside'},
 {'id': 'http://www.wikidata.org/entity/Q1211829',
  'Film': 'The Bad News Bears'}]